<a href="https://colab.research.google.com/github/ORCT/ds-section6-project/blob/main/binary_estimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0과 1을 분류하는 인공신경망 프로그래밍 

0과 1을 분류하는 인공신경망을 딥러닝 라이브러리를 사용하지 않고 구현하는 것이 목적이다.

세부사항 : 
- 순전파 과정만을 구현한다.
- 핵심 기능을 재사용 가능하도록 구현한다.

사용한 라이브러리 :
- numpy
- pandas
- csv
- matplotlib

In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [ ]:
def data_load(dir):
  '''
  데이터를 입력 경로에 따라 호출한다.
  '''
  return pd.read_csv(dir)

In [ ]:
def data_shuffle(df):
  '''
  data를 섞는다.
  np.permutation 메서드를 이용해 원본을 보존하면서 섞는다.
  '''
  return pd.DataFrame(np.random.permutation(df))

In [ ]:
def X_y_split(df):
  '''
  target label과 feature를 분리한다.
  '''
  target = 8
  X = df.drop(target, axis=1)
  y = df[target]
  return X, y

In [ ]:
def train_test_split(df):
  '''
  train, test를 나눈다.
  조건에 따라 test 사이즈를 4, 나머지를 train으로 한다.
  '''
  test_idx = np.random.randint(20, size=4)
  test = df.iloc[test_idx]
  train = df.drop(test_idx, axis=0)
  return train, test

In [ ]:
def init_w_b(df):
  '''
  가중치와 편향을 최초 랜덤값으로 초기화한다.
  랜덤값의 범위를 0 ~ 1 까지로 선정했다.
  '''
  weight = np.random.rand(df.shape[1]-1)
  bias = np.random.rand(1)
  return weight, bias

In [ ]:
def batch_split(df, batch_size=4):
  '''
  이미 셔플이 된 df(train data)가 입력으로 주어진다.
  batch_size는 조건에 따라 4로 고정한다.
  batch_size에 따라 나눠진 mini_batch를 담은 리스트를 반환한다.
  '''
  df = df.reset_index(drop=True)
  batchs = []
  for i in range(0, len(df), 4):
    batchs.append(df.iloc[i:i+4,:])
  return batchs

In [ ]:
def apply_sigmoid(data):
  '''
  출력층 활성화 함수로 Sigmoid를 사용한다.
  식 1/(1+e^(-x))
  임계치를 기준으로 분류한 1 또는 0 으로 이루어진 리스트를 반환한다.
  '''
  tmp = [1/(1+np.exp(1)**(-i)) for i in data]
  threshold = 0.5
  pred = [1 if i > threshold else 0 for i in tmp]
  return pred

In [ ]:
def cal_weight_sums(df, weight, bias):
  '''
  역전파가 없으므로 가중치의 변경이 일어나지 않는다.
  따라서 모든 배치에서 최초 가중치를 사용한다.
  '''
  weight_sums = []
  for i in range(len(df)):
    weight_sum = np.squeeze(np.sum(df.iloc[i, 0:8] * weight)) + bias[0]
    weight_sums.append(weight_sum)
  return weight_sums

In [ ]:
def cal_loss(pred, true):
  '''
  binary cross entropy loss 를 구한다.
  BCE 공식을 통해 구해진 값을 반환한다.
  log의 진수가 0이 되어 계산이 되지 않는 경우를 고려해 아주 작은 값을 더해준다.
  '''
  delta = 1e-7
  result = [-(true.iloc[i]*np.log(pred[i] + delta) + (1 - true.iloc[i])*np.log(1 - pred[i] + delta)) for i in range(len(pred))]
  return result

In [ ]:
def cal_accuracy(pred, true):
  '''
  Accuracy를 구한다.
  pred와 true를 비교해서 같으면 1을 아니면 0을 리스트에 추가한다.
  해당 리스트를 반환한다.
  '''
  acc_list = []
  for i in range(len(pred)):
    if pred[i] == true.iloc[i]:
      acc_list.append(1)
    else:
      acc_list.append(0)
  return acc_list

In [ ]:
def evaluate(df, weight, bias):
  '''
  1. 각 행의 가중합을 구한다.
  2. 구해진 가중합을 시그모이드에 집어넣는다.
  3. 나온 값을 통해 loss와 accuracy를 구한다.
  '''
  X, y = X_y_split(df)
  weight_sums = cal_weight_sums(X, weight, bias)
  y_pred = apply_sigmoid(weight_sums)
  loss = cal_loss(y_pred, y)
  accuracy = cal_accuracy(y_pred, y)
  return loss, accuracy

# 역전파(Back Propagation)

loss를 줄이는 방향으로 가중치를 업데이트하여 결정적으로 모델의 성능을 높이는데 기여, 순전파의 반대

- 먼저 loss를 줄이는 방향(여기서 loss는 BCE)


In [ ]:
def main():
  dir = 'binary_dataset.csv'
  df = data_load(dir)
  df = data_shuffle(df)
  train, test = train_test_split(df)
  batchs = batch_split(train)
  weight, bias = init_w_b(train)
  loss = []
  accuracy = []
  for i in batchs:
    l, a = evaluate(i, weight, bias)
    loss.extend(l)
    accuracy.extend(a)
  print(f'[Epoch 1] TrainData - Loss = {round(np.mean(loss), 3)}, Accuracy = {round(np.mean(accuracy), 3)}')

In [ ]:
main()

[Epoch 1] TrainData - Loss = 12.089, Accuracy = 0.25
